In [ ]:
!pip config set global.extra-index-url https://pip.repos.neuron.amazonaws.com
!pip install torch-neuron neuron-cc[tensorflow] "protobuf" torchvision!pip install boto3
!pip install transformers==4.34.0 

In [2]:
import torch 
import torch_neuron
from transformers import AutoTokenizer, AutoModel

/home/ubuntu/anaconda3/envs/yanolja/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
sequence_0 = "hello"
sequence_1 = "hi"

tokenized_sequence_pair = tokenizer.encode_plus(sequence_0,
                                                sequence_1,
                                                max_length=1024,
                                                padding='max_length',
                                                truncation=True,
                                                return_tensors='pt')

example_inputs = tokenized_sequence_pair['input_ids'], tokenized_sequence_pair['attention_mask']

# Let's compile the model from scatch

In [ ]:
base_model = AutoModel.from_pretrained('BAAI/bge-m3')

In [ ]:
compiled_model = torch.neuron.trace(base_model, example_inputs, strict=False)

In [ ]:
outputs = compiled_model(*example_inputs)

In [ ]:
outputs

컴파일이 완료된 모델을 저장합니다.

In [ ]:
compiled_model.save('compiled_model/model.pt')

저장된 모델이 잘 동작하는지 `torch.jit.load`로 다시 테스트합니다

In [6]:
model_path = "/home/ubuntu/yanolja/compiled_model/model.pt"
model_neuron = torch.jit.load(model_path)

In [7]:
outputs = model_neuron(*example_inputs)
outputs

{'last_hidden_state': tensor([[[-1.1719,  0.4141, -1.1562,  ...,  0.5625, -0.8984,  1.5391],
          [-0.1982, -0.4531, -0.6367,  ...,  1.3672, -0.5547,  1.6875],
          [-0.3027, -0.0093, -0.6602,  ...,  1.1484, -0.2051,  1.8594],
          ...,
          [-0.7305,  0.4590, -0.5352,  ...,  0.9219, -0.6914,  1.3750],
          [-0.7305,  0.4590, -0.5352,  ...,  0.9219, -0.6914,  1.3750],
          [-0.7305,  0.4590, -0.5352,  ...,  0.9219, -0.6914,  1.3750]]]),
 'pooler_output': tensor([[-0.9062, -0.0457,  0.3613,  ..., -0.4355,  0.2236, -0.2256]])}

In [34]:
from sentence_transformers import util

sentences = ["I'm happy", "I'm full of joy"]

tokenized_1 = tokenizer.encode_plus(sentences[0],max_length=1024,padding='max_length',truncation=True,return_tensors='pt')
tokenized_2 = tokenizer.encode_plus(sentences[1],max_length=1024,padding='max_length',truncation=True,return_tensors='pt')

inputs_1 = tokenized_1['input_ids'], tokenized_1['attention_mask']
inputs_2 = tokenized_2['input_ids'], tokenized_2['attention_mask']

embedding_1 = model_neuron(*inputs_1)
embedding_2 = model_neuron(*inputs_2)

sim = util.pytorch_cos_sim(embedding_1['pooler_output'], embedding_2['pooler_output'])

out_str = f'Similaritiy of two sentences are {round(sim[0][0].item(),3)}'
out_str

'Similaritiy of two sentences are 0.838'

In [35]:
torch.nn.functional.cosine_similarity(embedding_1['pooler_output'], embedding_2['pooler_output'])

tensor([0.8381])

준비된 모델 파일과 `inference.py`를 tar.gz로 압축해서 업로드합니다.

In [ ]:
# copy inference.py and requirements.txt to neuron_model/code
!cd compiled_model && tar -czvf ../model.tar.gz *

In [ ]:
!cd /home/ubuntu/yanolja
!aws s3 cp model.tar.gz s3://sagemaker-ap-northeast-2-236241703319/neuron-experiments/bge-m3/compiled-model/compiled_model.tar.gz